# TradeStation Futures Data Analysis

This notebook demonstrates how to work with the downloaded futures data.

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

DATA_DIR = Path('./data')

## 1. Load Data

In [ ]:
# List available symbols
symbols = [f.stem.replace('_1min', '') for f in DATA_DIR.glob('*_1min.parquet')]
print(f'Available symbols: {len(symbols)}')
print(symbols[:10], '...' if len(symbols) > 10 else '')

In [ ]:
# Load E-mini S&P 500
es = pd.read_parquet(DATA_DIR / 'ES_1min.parquet')
es['datetime'] = pd.to_datetime(es['datetime'])
es = es.set_index('datetime')

print(f'ES Data:')
print(f'  Date range: {es.index.min()} to {es.index.max()}')
print(f'  Total bars: {len(es):,}')
es.head()

## 2. Resample to Different Timeframes

In [ ]:
def resample_ohlcv(df, timeframe):
    """Resample OHLCV data to a different timeframe."""
    return df.resample(timeframe).agg({
        'open': 'first',
        'high': 'max',
        'low': 'min',
        'close': 'last',
        'volume': 'sum'
    }).dropna()

# Create different timeframes
es_5min = resample_ohlcv(es, '5min')
es_15min = resample_ohlcv(es, '15min')
es_1h = resample_ohlcv(es, '1H')
es_daily = resample_ohlcv(es, '1D')

print(f'5-min bars: {len(es_5min):,}')
print(f'15-min bars: {len(es_15min):,}')
print(f'Hourly bars: {len(es_1h):,}')
print(f'Daily bars: {len(es_daily):,}')

## 3. Basic Statistics

In [ ]:
# Daily returns
es_daily['return'] = es_daily['close'].pct_change()
es_daily['log_return'] = np.log(es_daily['close'] / es_daily['close'].shift(1))

print('ES Daily Return Statistics:')
print(f'  Mean return: {es_daily["return"].mean()*100:.4f}%')
print(f'  Std dev: {es_daily["return"].std()*100:.4f}%')
print(f'  Annualized vol: {es_daily["return"].std()*np.sqrt(252)*100:.2f}%')
print(f'  Sharpe ratio: {es_daily["return"].mean() / es_daily["return"].std() * np.sqrt(252):.2f}')
print(f'  Max drawdown: {(es_daily["close"] / es_daily["close"].cummax() - 1).min()*100:.2f}%')

## 4. Visualization

In [ ]:
# Plot daily close prices
fig, axes = plt.subplots(2, 1, figsize=(14, 8))

# Price chart
axes[0].plot(es_daily.index, es_daily['close'])
axes[0].set_title('ES Daily Close Price')
axes[0].set_ylabel('Price')
axes[0].grid(True, alpha=0.3)

# Volume chart
axes[1].bar(es_daily.index, es_daily['volume'], width=1, alpha=0.7)
axes[1].set_title('ES Daily Volume')
axes[1].set_ylabel('Volume')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 5. Multi-Symbol Analysis

In [ ]:
# Load multiple symbols
symbols_to_compare = ['ES', 'NQ', 'CL', 'GC']
data = {}

for sym in symbols_to_compare:
    filepath = DATA_DIR / f'{sym}_1min.parquet'
    if filepath.exists():
        df = pd.read_parquet(filepath)
        df['datetime'] = pd.to_datetime(df['datetime'])
        df = df.set_index('datetime')
        data[sym] = resample_ohlcv(df, '1D')
        print(f'{sym}: {len(data[sym]):,} daily bars')

In [ ]:
# Calculate correlations
returns = pd.DataFrame({sym: df['close'].pct_change() for sym, df in data.items()})
correlation = returns.corr()

print('\nReturn Correlations:')
print(correlation.round(3))

## 6. Export for Backtesting

In [ ]:
# Export to CSV for use in other tools
output_dir = Path('./csv_export')
output_dir.mkdir(exist_ok=True)

# Export 1-minute data
es.reset_index().to_csv(output_dir / 'ES_1min.csv', index=False)

# Export daily data
es_daily.reset_index().to_csv(output_dir / 'ES_daily.csv', index=False)

print(f'Exported to {output_dir}')

## 7. Simple Backtest Example

In [ ]:
# Simple moving average crossover strategy
def backtest_ma_crossover(df, fast=20, slow=50):
    """Backtest a simple MA crossover strategy."""
    df = df.copy()
    
    # Calculate MAs
    df['ma_fast'] = df['close'].rolling(fast).mean()
    df['ma_slow'] = df['close'].rolling(slow).mean()
    
    # Generate signals
    df['signal'] = np.where(df['ma_fast'] > df['ma_slow'], 1, -1)
    df['signal'] = df['signal'].shift(1)  # Avoid lookahead
    
    # Calculate returns
    df['return'] = df['close'].pct_change()
    df['strategy_return'] = df['signal'] * df['return']
    
    # Calculate cumulative returns
    df['cum_return'] = (1 + df['return']).cumprod()
    df['cum_strategy'] = (1 + df['strategy_return']).cumprod()
    
    return df

# Run backtest on daily data
results = backtest_ma_crossover(es_daily.copy())

print('MA Crossover Strategy Results:')
print(f'  Total Return (Buy & Hold): {(results["cum_return"].iloc[-1] - 1)*100:.2f}%')
print(f'  Total Return (Strategy): {(results["cum_strategy"].iloc[-1] - 1)*100:.2f}%')
print(f'  Strategy Sharpe: {results["strategy_return"].mean() / results["strategy_return"].std() * np.sqrt(252):.2f}')

In [ ]:
# Plot strategy performance
fig, ax = plt.subplots(figsize=(14, 6))

ax.plot(results.index, results['cum_return'], label='Buy & Hold', alpha=0.7)
ax.plot(results.index, results['cum_strategy'], label='MA Crossover', alpha=0.7)
ax.set_title('ES Strategy Comparison')
ax.set_ylabel('Cumulative Return')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()